In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Load data from CSV
file_path_women = r"C:\Users\Rishav Gupta\Desktop\CAPSTONE\India_Crime - Copy\42_District_wise_crimes_committed_against_women_2001_2012.csv"
df_women = pd.read_csv(file_path_women)

file_path_st = r"C:\Users\Rishav Gupta\Desktop\CAPSTONE\India_Crime - Copy\02_District_wise_crimes_committed_against_ST_2001_2012.csv"
df_st = pd.read_csv(file_path_st)

file_path_ipc = r"C:\Users\Rishav Gupta\Desktop\CAPSTONE\India_Crime - Copy\01_District_wise_crimes_committed_IPC_2001_2012.csv"
df_ipc = pd.read_csv(file_path_ipc)

# Rename columns to avoid SQL syntax errors
df_women.rename(columns={'STATE/UT': 'State_UT'}, inplace=True)
df_st.rename(columns={'STATE/UT': 'State_UT'}, inplace=True)
df_ipc.rename(columns={'STATE/UT': 'State_UT'}, inplace=True)

# Connect to SQLite database (or create it)
conn = sqlite3.connect('india_crime.db')
cursor = conn.cursor()

# Create table for Crimes Against Women
cursor.execute('''
CREATE TABLE IF NOT EXISTS Crimes_Against_Women (
    State_UT TEXT,
    District TEXT,
    Year INTEGER,
    Rape INTEGER,
    Kidnapping_and_Abduction INTEGER,
    Dowry_Deaths INTEGER,
    Assault_on_Women_with_Intent_to_outrage_her_Modesty INTEGER,
    Insult_to_the_Modesty_of_Women INTEGER,
    Cruelty_by_Husband_or_his_Relatives INTEGER,
    Importation_of_Girls INTEGER
)
''')

# Insert records into the table
df_women.to_sql('Crimes_Against_Women', conn, if_exists='replace', index=False)

# Create table for Crimes Against ST
cursor.execute('''
CREATE TABLE IF NOT EXISTS Crimes_Against_ST (
    State_UT TEXT,
    District TEXT,
    Year INTEGER,
    Murder INTEGER,
    Attempt_to_Murder INTEGER,
    Culpable_Homicide_not_amounting_to_Murder INTEGER,
    Rape INTEGER,
    Kidnapping_and_Abduction INTEGER,
    Dacoity INTEGER,
    Robbery INTEGER,
    Arson INTEGER
)
''')

# Insert records into the table
df_st.to_sql('Crimes_Against_ST', conn, if_exists='replace', index=False)

# Create table for IPC Crimes
cursor.execute('''
CREATE TABLE IF NOT EXISTS IPC_Crimes (
    State_UT TEXT,
    District TEXT,
    Year INTEGER,
    Murder INTEGER,
    Attempt_to_Murder INTEGER,
    Culpable_Homicide_not_amounting_to_Murder INTEGER,
    Rape INTEGER,
    Kidnapping_and_Abduction INTEGER,
    Dacoity INTEGER,
    Robbery INTEGER,
    Arson INTEGER
)
''')

# Insert records into the table
df_ipc.to_sql('IPC_Crimes', conn, if_exists='replace', index=False)


# Highest Number of Dacoity/Robbery
query_highest_dacoity = '''
SELECT State_UT, District, MAX(Dacoity) as Max_Dacoity
FROM Crimes_Against_ST
GROUP BY District
ORDER BY Max_Dacoity DESC
LIMIT 1
'''
highest_dacoity = pd.read_sql_query(query_highest_dacoity, conn)
print("Highest number of dacoity:", highest_dacoity)

# Districts with the Lowest Number of Murders
query_lowest_murders = '''
SELECT State_UT, District, MIN(Murder) as Min_Murder
FROM IPC_Crimes
GROUP BY District
ORDER BY Min_Murder ASC
'''
lowest_murders = pd.read_sql_query(query_lowest_murders, conn)
print("Lowest number of murders:", lowest_murders)

# Number of Murders in Ascending Order (District and Year-wise)
query_murders_ascending = '''
SELECT State_UT, District, Year, Murder
FROM IPC_Crimes
ORDER BY Murder ASC, District ASC, Year ASC
'''
murders_ascending = pd.read_sql_query(query_murders_ascending, conn)
print("Number of murders in ascending order:", murders_ascending)

# Create a new table for specific columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS Selected_IPC_Crimes (
    State_UT TEXT,
    District TEXT,
    Year INTEGER,
    Murder INTEGER,
    Attempt_to_Murder INTEGER,
    Rape INTEGER
)
''')

# Insert selected columns
selected_columns_df = df_ipc[['State_UT', 'DISTRICT', 'YEAR', 'MURDER', 'RAPE']]
selected_columns_df.to_sql('Selected_IPC_Crimes', conn, if_exists='replace', index=False)

# Highest Number of Murders Year-wise
query_highest_murders = '''
SELECT State_UT, YEAR, DISTRICT, MAX(MURDER) as Max_MURDER
FROM Selected_IPC_Crimes
GROUP BY State_UT, YEAR, DISTRICT
ORDER BY Max_MURDER DESC
'''
highest_murders = pd.read_sql_query(query_highest_murders, conn)
print("Highest number of murders year-wise:", highest_murders)

# Analysis of Districts Appearing 3 or More Years
df_highest_murders = pd.read_sql_query(query_highest_murders, conn)
districts_appearing_3_or_more = df_highest_murders['DISTRICT'].value_counts()
districts_appearing_3_or_more = districts_appearing_3_or_more[districts_appearing_3_or_more >= 3]
analysis_df = df_highest_murders[df_highest_murders['DISTRICT'].isin(districts_appearing_3_or_more.index)]
analysis_df_sorted = analysis_df.sort_values(by=['State_UT', 'DISTRICT', 'YEAR', 'Max_MURDER'], ascending=[True, True, True, False])
print("Districts appearing 3 or more years:", analysis_df_sorted)

# Plot the data
plt.figure(figsize=(14, 8))
sns.barplot(data=analysis_df_sorted, x='DISTRICT', y='Max_MURDER', hue='YEAR', dodge=True)
plt.title('Districts with Highest Murders in Each Year')
plt.xlabel('DISTRICTS')
plt.ylabel('Number of Murders')
plt.xticks(rotation=90)
plt.legend(title='YEAR')
plt.show()


Highest number of dacoity:   State_UT DISTRICT  Max_Dacoity
0  GUJARAT    TOTAL           29
Lowest number of murders:               State_UT          DISTRICT  Min_Murder
0    ARUNACHAL PRADESH             ANJAW           0
1                ASSAM              BIEO           0
2                ASSAM            C.I.D.           0
3             DELHI UT               CAW           0
4           TAMIL NADU      CHENNAI RLY.           0
..                 ...               ...         ...
803          KARNATAKA  BANGALORE COMMR.         201
804        MAHARASHTRA     MUMBAI COMMR.         203
805        MAHARASHTRA            MUMBAI         212
806              BIHAR             PATNA         248
807           DELHI UT    DELHI UT TOTAL         455

[808 rows x 3 columns]
Number of murders in ascending order:                State_UT DISTRICT  YEAR  MURDER
0     ARUNACHAL PRADESH    ANJAW  2009       0
1     ARUNACHAL PRADESH    ANJAW  2012       0
2                 ASSAM     BIEO  2012    